In [5]:
import time
import json5
import requests
import pandas as pd

In [6]:
url = 'https://www.indeed.com/jobs?q=machine%20learning&l=remote&explvl=entry_level&fromage=7'

## The following line needs to be changed every time you run this
Open the url above in your browser and look for the number of jobs listed

In [7]:
job_count = 96

In [8]:
# create an empty list to store the jobs
jobs = []

# get the jobs from indeed
for page in range(job_count//50+1):
    err_count = 0 # this is used in case multiple requests are needed
    
    # get the page
    r = requests.get(url+'&start={}&limit=50'.format(page*50))
    
    # make multiple requests if needed
    while r.status_code != 200 and err_count < 5:
        time.sleep(5)
        r = requests.get(url+'&start={}&limit=50'.format(page*50))
        err_count += 1
        
    # find the job information, Indeed stores the information in Javascript objects in a list called jobmap   
    for i in range(50):
        start = r.text.find('jobmap[{}]'.format(i))
        if start==-1: #yes, I know, this isn't the most efficient way
            continue
        end = r.text.find('jobmap[{}]'.format(i+1))
        
        # get the start and end point of the javascript object
        s = r.text.find('{',start)
        e = r.text.find('}',s)+1
        
        # get the javascript object string
        job = r.text[s:e]
        p = job.find(': ')
        
        while p != -1: # just tidying a little, this isn't needed
            job = job.replace(': ',':')
            p = job.find(': ')
        
        # converting the javascript object to a python dictionary
        job = json5.loads(job)
        
        #saving the job 
        jobs.append(job)
    time.sleep(5)

In [9]:
df = pd.DataFrame(jobs)
df.head().transpose()

,0,1,2,3,4
jk,8f236d29074939b5,42173e26ab1dae81,aed1870930baf6d0,322733aec77dbe12,641881963c477817
efccid,f10f7a5d7fa4532a,7cb49319a64d4861,239991bba6d4fdac,a718462517070ed0,8a255221c92f37ed
srcid,eaf52f0ece9ad57f,3c432b0faa67efac,44daf54be0d6581a,90a58d6fd895079d,db00c3064ae26945
cmpid,7b2651385d36b6df,060bf3257ce4e3a1,48a8b92e8413a2d8,d6ef41e202aa2c0b,a9491f83c147c973
num,0,1,2,3,4
srcname,Indeed,Indeed,Indeed,Indeed,Indeed
cmp,Kalibri Labs,Real Talent Acquisition Group,Bald Einstein,Indeed,Fellowship.ai
cmpesc,Kalibri Labs,Real Talent Acquisition Group,Bald Einstein,Indeed,Fellowship.ai
cmplnk,/q-Kalibri-Labs-l-remote-jobs.html,/q-Real-Talent-Acquisition-Group-l-remote-jobs...,/q-Bald-Einstein-l-remote-jobs.html,/q-Indeed-l-remote-jobs.html,/q-Fellowship.ai-l-remote-jobs.html
loc,Remote,Remote,Remote,Remote,Remote


In [10]:
df['job_link']='https://www.indeed.com/viewjob?jk='+df.jk

In [12]:
df.rename(columns={'cmp':'company'},inplace=True)

In [14]:
df = df[['title','company','job_link','country']].copy()
df.head()

,title,company,job_link,country
0,Junior Data Scientist,Kalibri Labs,https://www.indeed.com/viewjob?jk=8f236d290749...,US
1,Machine Learning Support Engineer,Real Talent Acquisition Group,https://www.indeed.com/viewjob?jk=42173e26ab1d...,US
2,Machine Learning Engineer,Bald Einstein,https://www.indeed.com/viewjob?jk=aed1870930ba...,US
3,Staff Product Scientist - Monetization,Indeed,https://www.indeed.com/viewjob?jk=322733aec77d...,US
4,Machine Learning Remote Fellowship,Fellowship.ai,https://www.indeed.com/viewjob?jk=641881963c47...,US


In [15]:
df.to_csv('jobs.csv',index=False)